# Fake News Prediction

Ref: [Sidhardhan's ML projects](https://www.youtube.com/watch?v=nacLBdyG6jE&list=PLfFghEzKVmjvuSA67LszN1dZ-Dd_pkus6&index=4)

Author: Dathabase

> ### **Aim:**
To create a system that can predict whether a piece of news is fake or real using logistic regresssion

> ### **Workflow:**

1. Get news data and respective labels (fake/real)
1. Data pre-processing (convert text into meaningful numbers so that the computer can understand the inputs)
1. Train, Test, Split
1. Use logistic regression model (supervised learning)
1. Train logistic regression model to identify unreliable news articles

In [1]:
# importing required modules

import pandas as pd
import numpy as np
import re # regular expressions; useful for searching text in documents
from nltk.corpus import stopwords # natural language toolkit; to remove articles (a, and, the)
from nltk.stem.porter import PorterStemmer # takes a word and returns its root word
from sklearn.feature_extraction.text import TfidfVectorizer # convert text to feature vectors i.e. numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# list of english words that don't add much value to the dataset
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## **Data Collection and Pre-processing**

This dataset was taken from kaggle's community prediction competition [(link)](https://www.kaggle.com/c/fake-news/data?select=train.csv)



test.csv: A testing training dataset with all the same attributes at train.csv without the label.

submit.csv: A sample submission that you can

In [4]:
news_df = pd.read_csv('news_data.csv')
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Columns:
- `id`: unique id for a news article
- `title`: the title of a news article
- `author`: author of the news article
- `text`: the text of the article; could be incomplete
- `label`: a label that marks the article as potentially unreliable. (1 = fake news; 0 = real news)

In [5]:
print(f"The dataframe has {news_df.shape[0]} rows and {news_df.shape[1]} columns")

The dataframe has 20800 rows and 5 columns


In [6]:
# check for missing values
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Apart from the `id` and `label` columns, there is a good amount of important information missing from the other columns. However considering the whole dataset, max 10% of the data is missing; hence it would be better to drop the rows where the values are missing

In [7]:
# replace null values with empty string
news_df.fillna('', inplace=True)

For this project, `title` and `author` are sufficient features to help train the model - `text` contains long paragraphs and so can be time consuming to convert it to numbers

In [8]:
# merge news title and author name
news_df['content'] = news_df['author']  + " " + news_df['title']

In [9]:
news_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


**Stemming:** Process of reducing a word to its root. (eg: actor, actress, acting ---> act)

Use `PorterStemmer` to perform the stemming process

In [10]:
port_stem = PorterStemmer()

In [11]:
def stemming(text):
  '''
  function defined that takes in some text, selects only the alphabets 
  (no stopwords/punctatuations/numbers) and then reduces them to the root words
  '''
  stemmed_text = re.sub('[^a-zA-Z]',' ',text) # replace numbers, punctuations, etc by spaces
  stemmed_text = stemmed_text.lower().split() # convert alphabets to lowercase and split into a list
  stemmed_text = [port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')] # exclude stopwords and apply stemming procedure
  stemmed_text = ' '.join(stemmed_text) # join the words
  return stemmed_text

In [12]:
news_df['content'] = news_df['content'].apply(stemming)

Separate the input data, X i.e. **features** (`content`) and the data to be predicted, Y i.e. **labels** (`label`)

In [16]:
X = news_df.loc[:,'content'].values
Y = news_df.loc[:,'label'].values

Convert textual data to numerical data using `TfidfVectorizer` (`Tf` stands for Term frequency and `idf` stands for Inverse document frequency)
- counts number of times a particular word repeats in a document
- assigns a particular numerical value to that word (greater frequency => low importance)

In [20]:
vec = TfidfVectorizer()
vec.fit(X)
X = vec.transform(X)

## **Train, Test, Split**

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                    random_state = 2,
                                                    stratify = Y,
                                                    test_size = 0.2)

Parameters:
- `X_train` = training data; `Y_train` = corresponding labels of the training data
- `X_test` = testing data; `Y_test` = corresponding labels of the testing data
- 20% of data will be in the testing set => `test_size = 0.2`
- should spilt the data similarly whenever the function is called => `random_state = 2`

## **Model Training and Evaluation**

In [110]:
news_log_model = LogisticRegression()

In [111]:
# fit training data
news_log_model.fit(X_train, Y_train)

LogisticRegression()

In [112]:
# accuracy on training data
X_train_pred = news_log_model.predict(X_train)
accuracy_X_train_pred = accuracy_score(X_train_pred, Y_train)
print(f"Accuracy on Training data: {accuracy_X_train_pred*100:.1f}%")

Accuracy on Training data: 98.7%


In [113]:
# accuracy on testing data
X_test_pred = news_log_model.predict(X_test)
accuracy_X_test_pred = accuracy_score(X_test_pred, Y_test)
print(f"Accuracy on Testing data: {accuracy_X_test_pred*100:.1f}%")

Accuracy on Testing data: 97.9%


Make predictions using testing data...

In [126]:
# choose random index
def rand_num(min_val, max_val):
  import random
  i = random.randint(min_val, max_val)
  return i

rand_idx = rand_num(0, X_test.shape[0]-1)

In [127]:
# make predictions on test data
X_test_pred = news_log_model.predict(X_test[rand_idx])

if X_test_pred[0] == 0:
  print("The news is real")
elif X_test_pred[0] == 1:
  print("The news is fake")
else:
  print("Invalid prediction.")

if X_test_pred[0] == Y_test[rand_idx]:
  print("This prediction is correct :)")
else:
  print("This prediction is incorrect :(")


The news is real
This prediction is correct :)
